<a href="https://colab.research.google.com/github/janakurrek/mnli/blob/master/BiLSTM%2BAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import json
import torch
import wget
import os
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torchtext.utils import download_from_url

print('Downloading dataset...')

datapath = "/content/drive/My Drive/colab-data/kurrek.2020.slur-corpus.tiny.csv"
pd.read_csv(datapath).head()

,label,body_clean
0,0,Fuck that I dont wanna watch tranny porn.
1,0,Opie just wanted to have a good time. The same...
2,0,Fuck that faggot Fallon. Fucking sissy boy act...
3,0,BbBB...b.b..b.bb but OP's a faggot
4,1,Who even uses the word tranny except for trans...


**Pay "Attention" to Your Context**

https://www.aclweb.org/anthology/W19-3508.pdf

https://github.com/tuhinjubcse/ALW3-ACL2019

https://www.aclweb.org/anthology/N16-1174/

https://github.com/uvipen/Hierarchical-attention-networks-pytorch/blob/master/src/utils.py


*   Stacked Bi-LSTM models outperformed the simple Bi-LSTM models
*   Best performing model on abusive language datasets is the stacked Bi-LSTM with contextual attention



In [15]:
"""
TEXT = data.Field(
    tokenize='spacy', 
    lower=True
)

LABELS = data.Field(
    sequential=False,
    use_vocab=False,
    dtype=torch.float
)

dataset = data.TabularDataset(
    path=os.path.join(datapath),
    format='csv',
    fields={'body_clean': ('text', TEXT), 'label': ('labels', LABELS)}
)

D_trn, D_val, D_tst = dataset.split(split_ratio=[0.7, 0.1, 0.2], random_state=random.getstate())


# load fastext simple embedding with 300d
# !pip3 install --upgrade torchtext

vectors – one of or a list containing instantiations of the GloVe, CharNGram, or Vectors classes.
Alternatively, one of or a list of available pretrained vectors: charngram.100d fasttext.en.300d
fasttext.simple.300d glove.42B.300d glove.840B.300d glove.twitter.27B.25d glove.twitter.27B.50d
glove.twitter.27B.100d glove.twitter.27B.200d glove.6B.50d glove.6B.100d glove.6B.200d glove.6B.300d

TEXT.build_vocab(D_trn, vectors='glove.6B.50d')
LABELS.build_vocab(D_trn, vectors='glove.6B.50d')

# get the vocab instance
# Unique tokens in text vocabulary: 35577 (glove 6B 50d)
# Unique tokens in text vocabulary: 35577 (fasttext.simple.300d)

print(f"Unique tokens in text vocabulary: {len(TEXT.vocab)}")

"""

from torchtext import data
from torchtext.vocab import GloVe
import random

MAX_INPUT_LEN = 320

# tokenizer settings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

unk_token_idx = tokenizer.unk_token_id
pad_token_idx = tokenizer.pad_token_id

def tokenize_and_cut(sentence, max_input_length = 320):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length - 2]
    return tokens

# load text inputs and labels
TEXT = data.Field(batch_first=True, use_vocab=False, tokenize=tokenize_and_cut,
                  preprocessing=tokenizer.convert_tokens_to_ids,
                  pad_token=pad_token_idx, unk_token=unk_token_idx)

LABELS = data.Field(
    sequential=False,
    dtype=torch.float
)

dataset = data.TabularDataset(
    path=os.path.join(datapath),
    format='csv',
    fields={'body_clean': ('text', TEXT), 'label': ('labels', LABELS)}
)

D_trn, D_val, D_tst = dataset.split(split_ratio=[0.7, 0.1, 0.2], random_state=random.getstate())

# build vocabulary and count tokens
TEXT.build_vocab(D_trn)
LABELS.build_vocab(D_trn)

print(f"Unique tokens in text vocabulary: {len(TEXT.vocab)}")

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

Unique tokens in text vocabulary: 18579


In [20]:
import torch.nn as nn
import math

"""Hyperparameters"""
# training
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
BATCH_SIZE = 128

# model hyperparameters
OUTPUT_SIZE = 2 #  binary classification
EMBED_SIZE = MAX_INPUT_LEN
HIDDEN_SIZE = 1024 
NUM_LAYERS = 2 # benchmark did 4
DROPOUT = 0.5
VOCAB_SIZE = len(TEXT.vocab)

# gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

"""Model"""
class attbilstm(nn.Module):
    def __init__(self, vec=None):
        super().__init__()
        self.vocab_size = VOCAB_SIZE
        self.hidden_dim = HIDDEN_SIZE
        self.batch_size = BATCH_SIZE
        self.emb_dim = EMBED_SIZE
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBED_SIZE)
        if vec is not None:
            self.embedding.weight.data.copy_(vec) # load pretrained
        self.encoder = nn.LSTM(EMBED_SIZE, EMBED_SIZE, num_layers=NUM_LAYERS, bidirectional=True, dropout=DROPOUT)
        self.fc = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
        self.dropout = nn.Dropout(DROPOUT)
    
    def attnetwork(self, encoder_out, final_hidden):
        hidden = final_hidden.squeeze(0)
        attn_weights = torch.bmm(encoder_out, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        new_hidden = torch.bmm(encoder_out.transpose(1,2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return new_hidden
    
    def forward(self, sequence):
        emb_input = self.embedding(sequence)    
        inputx = self.dropout(emb_input)
        output, (hn, cn) = self.encoder(inputx)
        fbout = output[:, :, :self.hidden_dim]+ output[:, :, self.hidden_dim:] # sum bidir outputs F+B
        fbout = fbout.permute(1,0,2)
        fbhn = (hn[-2,:,:]+hn[-1,:,:]).unsqueeze(0)
        attn_out = self.attnetwork(fbout, fbhn)
        logits = self.fc(attn_out)
        return logits

In [21]:
import torch.optim as optim

def accuracy(pred, label):
    prob, idx = torch.max(pred,1)
    precision = (idx==label).float().mean()
    if gpu:
        accuracy.append(precision.data.item())
    else:
        accuracy.append(precision.data.numpy()[0])
    return np.mean(accuracy)

def cat_accuracy(pred, label):
    max_preds = pred.argmax(dim=1, keepdim=True)
    correct = max_preds.squeeze(1).eq(label)
    correct = correct.sum().unsqueeze(0)
    bs = torch.LongTensor([label.shape[0]]).to(device)
    acc = correct.item() / bs.item()
    return acc

def train(model, it, lossf, optimizer):
    model.train()
    ep_loss = 0.0
    ep_acc = 0.0
    for b in tqdm(it):
        optimizer.zero_grad()
        seq, label = b.text, b.labels
        pred = model(seq)
        loss = lossf(pred, label)
        acc = cat_accuracy(pred, label)
        loss.backward()
        optimizer.step()
        ep_loss += loss.item()
        ep_acc += acc
    return ep_loss/ len(it), ep_acc/ len(it)

def evaluate(model, it, lossf):
    model.eval()
    ep_loss = 0.0
    ep_acc = 0.0
    with torch.no_grad():
	    for b in it:
	        seq, label = b.text, b.labels
	        pred = model(seq)
	        loss = lossf(pred, label)
	        acc = cat_accuracy(pred, label)
	        ep_loss += loss.item()
	        ep_acc += acc
    return ep_loss/ len(it), ep_acc/ len(it)

train_it, valid_it, test_it = data.BucketIterator.splits(
    (D_trn, D_val, D_tst), 
     batch_size = BATCH_SIZE, # (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
     sort_within_batch = True,
     sort_key = lambda x: len(x.text), # minimize padding
     # device = device,
     repeat = False)

# pretrained_vec = TEXT.vocab.vectors
model = attbilstm() # vec=pretrained_vec
lossf = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# model.to(device)

for ep in range(NUM_EPOCHS):
    print ('training epoch...', ep)
    tr_loss, tr_acc = train(model, train_it, lossf, optimizer)
    print ('TRAIN: loss %.2f acc %.1f' % (tr_loss, tr_acc))
    vl_loss, vl_acc = evaluate(model, valid_it, lossf)
    print('VALID: loss %.2f acc %.1f' % (vl_loss, vl_acc))

te_loss, te_acc = evaluate(model, test_it, lossf)
print('TEST: loss %.2f acc %.1f' % (te_loss, te_acc))

training epoch... 0


/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


IndexError: ignored

In [9]:

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

AttributeError: ignored

In [14]:
from transformers import BertModel
bert_model = BertModel.from_pretrained('bert-base-uncased')

NameError: ignored